import libraries

In [2]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm
from functools import reduce

import custom functions

In [ ]:
# Specify the full path to the module (including the filename)
module_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/parser.py'

# Import the module using importlib (a workaround if the standard import doesn't work)
import importlib.util
spec = importlib.util.spec_from_file_location("parser", module_path)
parser = importlib.util.module_from_spec(spec)
spec.loader.exec_module(parser)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1 Data collection

## 1.1. Get the list of master's degree courses

In [ ]:
def extract_masters(this_url):
    result_url = requests.get(this_url)
    result_soup = BeautifulSoup(result_url.text, 'html.parser')
    result_links = result_soup.find_all('a', {'class': 'courseLink'})
    result_list = []
    for item in result_links:
        result_list.append(item['href'])
    return result_list

In [ ]:
# Output file path
output_file_path = 'msc_urls.txt'
# Loop through the first 400 pages and write results to the output file
with open(output_file_path, 'a') as output_file:
    for page_number in range(1, 401):
        page_url = f'https://www.findamasters.com/masters-degrees/msc-degrees/?PG={page_number}'
        page_results = extract_masters(page_url)
        for url in page_results:
            output_file.write(f'{url}\n')
        time.sleep(1)

In [ ]:
with open(output_file_path, 'r') as file:
    lines = file.readlines()
    number_of_lines = len(lines)

print(f'The file {output_file_path} contains {number_of_lines} rows.')

The file msc_urls.txt contains 6000 rows.


## 1.2. Crawl master's degree pages

In [ ]:
# Function to download HTML from a URL with prefix and save it to a file
def download_and_save_html(url, output_path):
    full_url = 'https://www.findamasters.com/' + url
    try:
        response = requests.get(full_url)
        if response.status_code == 200:
            with open(output_path, 'w', encoding='utf-8') as html_file:
                html_file.write(response.text)
        else:
            print(f"Failed to download {full_url}. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {full_url}: {e}")

# Function to create a directory if it doesn't exist
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

# Main directory for downloaded HTML files
main_directory = 'downloaded_pages'
create_directory(main_directory)

# Iterate through the URLs and download the HTML
with open('msc_urls.txt', 'r') as file:
    for index, url in enumerate(file, start=1):
        # Remove whitespaces and newline characters from the URL
        url = url.strip()

        # Generate the directory path for the current page
        page_directory = os.path.join(main_directory, f'page_{(index - 1) // 15 + 1}')
        create_directory(page_directory)

        # Generate the output HTML file path
        output_path = os.path.join(page_directory, f'course_{index}.html')

        # Download the HTML and save it to the file
        download_and_save_html(url, output_path)
        time.sleep(3)

print("Download complete for all pages.")

Download complete for all pages.


#[1.3] Parse downloaded pages



At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format as desiried.

#Access the stored HTML pages

In [ ]:
%pip install google-colab-shell
# import the module once
from google_colab_shell import getshell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=80e086c86d11d45d2f29c9a491589816fae1262d0347b13dc5ea33ecab08cf12
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


used command

---mettere modalità python--
git clone https://github.com/marinazanoni/nome_provvisorio.git

after accesing the repository

In [ ]:
def parser(html_page):

    # Define your default values here
    default_values = {
    'courseName': None,
    'universityName': None,
    'facultyName': None,
    'isItFullTime': None,
    'description': None,
    'startDate': None,
    'fees': None,
    'modality': None,
    'duration': None,
    'city': None,
    'administration': None,
    'country': None,
    'url': None
}

    # Create a defaultdict with default values
    info = defaultdict(lambda: default_values)

   # Extract the text (HTML)
    with open(html_page, 'r', encoding='utf-8') as file:
        html_content = file.read()
    page_soup = BeautifulSoup(html_content, 'html.parser')

    # COURSE NAME
    page_links = page_soup.find_all('h1', {'class':'text-white course-header__course-title'})
    if page_links:
        first_h1_text = page_links[0].text
        info['courseName'] = str(first_h1_text)
    #else:
        # Handle the case when no 'h1' with 'data-permutive-title' is found
    #    print("No course name found.")
    #courseName = str(first_h1_text)

    # UNIVERSITY NAME
    # Extracting our tag of interest for the Name
    page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
    # Extracting the name of the course as string and print it
    info['universityName'] = str(page_links[0].contents[0])

    # FACULTY NAME
    page_links = page_soup.find_all('a', {'class': 'course-header__department'})
    info['facultyName'] = str(page_links[0].contents[0])

    # FULL TIME
    page_links = page_soup.find_all('a', {'class': 'inheritFont'})
    info['isItFullTime'] = str(page_links[0].contents[0])

    # SHORT DESCRIPTION
    page_links = page_soup.find('div', {'id': 'Snippet'})
    info['description'] = str(page_links.get_text(separator='\n').strip())

    # STARTING DATE
    page_links = page_soup.find('span', {'class': 'key-info__start-date'})
    info['startDate'] = str(page_links.get_text())

    # FEES
    page_links = page_soup.find('a', {'class': 'noWrap inheritFont'})
    page_links = page_soup.find('div', {'class': 'course-sections__fees'})
    if page_links:
        fees_text = page_links.get_text(separator='\n').strip()
        # Remove "Fees" from the text content
        info['fees'] = fees_text.replace('Fees', '').strip()


    # MODALITY
    page_links = page_soup.find('span', {'class': 'key-info__content key-info__qualification py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
    # Get all elements within the span using find_all
    elementsWithinSpan = page_links.find_all('a')
    info['modality'] = ' '.join([element.text.strip() for element in elementsWithinSpan])

    # DURATION
    page_links = page_soup.find('span', {'class':'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
    info['duration']=str(page_links.text)

    # CITY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
    info['city']=str(page_links.text)

    # ADMINISTRATION
    page_links = page_soup.find('span', {'class':'course-header__online-flag badge bg-white p-2 h6 shadow-sm mr-1'})
    if page_links == None:
        info['administration'] = "On Campus"
    else:
        info['administration']=str(page_links.text)

    # COUNTRY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
    info['country']=page_links.text

    #URL
    page_links = page_soup.find('link')
    info['url'] = page_links.get('href')


    return(pd.DataFrame([info]))


*Searching for a tipical hidden file and delete it in order not to have it when exctracting the information from the folder*

In [ ]:
parser('/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_1/course_1.html')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
file_path_to_remove = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/.DS_Store'
# Delete the hidden file
os.remove(file_path_to_remove)
print(f"The file {file_path_to_remove} has been deleted.")

In [ ]:
len(parsed_dfs)

5979

*Applying the function to our html pages*

In [ ]:
# Specifing the path of the folder
folder_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages'

# List all files in the folder
files = os.listdir(folder_path)

parsed_dfs = []

# Iterate through all folders and subfolders using os.walk
for folder_path, _, file_names in os.walk(folder_path):
    # Check if there are files in the current folder
    if file_names:
        # Iterate through each file in the current folder
        for file_name in file_names:
            file_path = os.path.join(folder_path, file_name)

            # Store the information only is the dictionary is not empty (has at list a name course)
            try:
                # Parse the file and append the result to the list
                parsed_df = parser(file_path)
                parsed_dfs.append(parsed_df)
            except Exception as e:
                # Print the file path when an exception occurs
                print(f"Error parsing file: {file_path}")
                # print(f"Error details: {e}")

# Concatenate all DataFrames in the list
concatenated_df = pd.concat(parsed_dfs, ignore_index=True)


Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_118/course_1765.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_119/course_1772.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_128/course_1910.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_140/course_2086.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2929.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2931.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_215/course_3213.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_291/course_4357.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_293/course_4395.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_2

In [ ]:
concatenated_df['url'][3]

'https://www.findamasters.com/masters-degrees/course/applied-economics-banking-and-financial-markets-online-msc/?i280d8352c56675'

In [ ]:
# Display the concatenated DataFrame
concatenated_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
concatenated_df.shape

(5979, 13)

In [ ]:
print(6000-5979, 'were not valid pages')

21 were not valid pages


In [ ]:
parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   courseName      1 non-null      object
 1   universityName  1 non-null      object
 2   facultyName     1 non-null      object
 3   isItFullTime    1 non-null      object
 4   description     1 non-null      object
 5   startDate       1 non-null      object
 6   fees            1 non-null      object
 7   modality        1 non-null      object
 8   duration        1 non-null      object
 9   city            1 non-null      object
 10  administration  1 non-null      object
 11  country         1 non-null      object
 12  url             1 non-null      object
dtypes: object(13)
memory usage: 232.0+ bytes


In [ ]:
# Specify the path where you want to save the .tsv file
tsv_file_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv'

# Save the DataFrame to a .tsv file
concatenated_df.to_csv(tsv_file_path, sep='\t', index=False)

print(f".tsv file saved at: {tsv_file_path}")

.tsv file saved at: /content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv


### *saving singularly the information about each master (html page)*

In [ ]:

# Iterate through each DataFrame in the list
for index, parsed_df in enumerate(parsed_dfs):
    # Iterate through each row in the DataFrame
    for row_index, row in parsed_df.iterrows():
        # Replace NaN values with empty strings
        single_row = row.fillna(' ')
        # Specify the path where you want to save the .tsv file for the current row
        tsv_file_path = f'/content/drive/MyDrive/HM3-ADM/HW3_ADM/Courses/course_{index}.tsv'
        # Save the single row DataFrame to a .tsv file
        with open(tsv_file_path, 'w') as file:
            file.write('\t'.join(single_row))
        # Uncomment to see all the tsv files
        #print(f".tsv file for row {index}, row {row_index} saved at: {tsv_file_path}")

#Preproccessing

Uploading the file created after parsing the information

In [4]:
df = pd.read_csv('/content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv',sep='\t')
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Linguistics - MSc,University of Glasgow,College of Arts & Humanities,Full time,This Masters focuses on how linguistic researc...,September,Please see the university website for further ...,MSc,1 year full-time; 2 years part-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import *

# Download the stopwords dataset if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#[2.0.0]

## Stemming

In [6]:
# Make a copy of the original DataFrame
processed_df = df.copy()

# stemmer
stemmer = PorterStemmer()

processed_df['description'] = processed_df.description.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.split(' ')]))

##Lower, removing punctuation and stepwords

In [7]:
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and punctuation from a text
def clean(text):
    """
    The following function returns the filtered element for each column of a dataframe.
    Filtering operation consists in removing punctuation and removing stopwords given text with lower case
    """
    words = word_tokenize(str(text))
    # Remove punctuation using NLTK and string.punctuation
    filtered_words = [word for word in words if word not in string.punctuation + "'’...?'+,-‘“”„•…›€✓"]
    # Remove stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Selecting columns
text_columns = processed_df.select_dtypes(include='object').columns
# Lower
processed_df[text_columns] = processed_df[text_columns].apply(lambda x: x.str.lower() if x.dtype == 'O' else x)
# Applying clean function to all the valid columns
processed_df[text_columns] = processed_df[text_columns].applymap(clean)


we verify which element we're filtering to check and think if something else need to be considered. Checking buy some outuput of the stemmed words we see some additional punctuation we don't want to consider which was added within the clean function.

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

We can know appreciate the differences of the two datasets (before and after preprocessing). Of our interesting will be the columns **fees** and **description**.

In [ ]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,air quality solutions msc,university leeds,institute transport studies,full time,7 million peopl estim die everi year due harm ...,september,"uk £12,500 total international £28,750 total",msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
1,analytical toxicology msc,king college london,faculty life sciences medicine,full time,analyt toxicolog msc uniqu studi cours integr ...,see course,please see university website information fees...,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
2,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,comput scienc foundat mani excit cutting-edg t...,september january,please see university website information fees...,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
3,applied economics banking financial markets on...,university bath,university bath online,part time,polit uncertainti financ recruit demands econo...,september january,cost per 10 credits £722 10 alumni discount pa...,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...
4,applied linguistics msc,university glasgow,college arts humanities,full time,thi master focus linguist research help solv l...,september,please see university website information fees...,msc,1 year full-time 2 years part-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...


#[2.0.1]

 ## we want the field fees to collect numeric information


We consider only *fees* column and deepen what we find

In [98]:
raw_fees= pd.DataFrame(processed_df['fees'])

In [99]:
raw_fees.head()

,fees
0,"uk £12,500 total international £28,750 total"
1,please see university website information fees...
2,please see university website information fees...
3,cost per 10 credits £722 10 alumni discount pa...
4,please see university website information fees...


In [ ]:
raw_fees['fees'].unique()

there are lot of missings data or more complex information we need to filter. To filter we follow the given guidline:


> in case of multiple information, retrieve only the highest fees. This suits perfectly the following example
>>*'uk £13,000 total international £29,000 total',
       '*

>In this other case will still apply, altough we're losing information so we will end up overestimating or underestimating the price. We decide like not to drop these value as can theorically give a (loose) range. Some general garbage are typically known when wanting to buy a master


>>*'eu fee course €10,820 €5,475 year 1 part-time €5,475 year 2 part-time cipd subscription fee €190 included student fee. non-eu fee course €18,700 includes cipd subscription fee €190 paid student',*


In [101]:
pattern = r'(?P<symbol>[^\d\s])(?P<value>\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'

def return_cost(stri):
    """
    This function returns a string built up by the currency and
    the maximum number it appears in the string given in input
    """
    # Search for the patterns in the string (currency, values)
    match = re.findall(pattern, stri)

    if not match:
        return None  # Return None if no currency symbols and values are found

    # Store all values
    result = [f"{currency[0]}{currency[1].replace(',', '')}" for currency in match]

    # Convert to a common currency format
    converted_list = [convert_to_common_currency(elem) for elem in result]

    # Exclude None values before finding the maximum
    non_none_values = [value for value in converted_list if value is not None]

    if not non_none_values:
       return None  # Return None if all values are None

    # Extract the maximum
    max_v = max(non_none_values)
    return max_v

text_columns = processed_df.select_dtypes(include='object').columns

# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['fees'] = raw_fees['fees'].apply(lambda x: return_cost(x) if x is not None else None)


###here we have filter fees column with only one cost each row, where available

In [102]:
raw_fees.head()

,fees
0,35739.59
1,NaN
2,NaN
3,897.53
4,NaN


In [84]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY'
    # Add more currency symbols as needed
}

# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency(price, target_currency='USD'):
    try:
        # Extract the currency symbol from the string
        currency_symbol = price[0]

        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(price[1:].replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)
        #return currency_symbol

    except Exception as e:
       return None


In [50]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY'
    # Add more currency symbols as needed
}

# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency(row, target_currency='USD'):
    if pd.isna(row['fees']):
        return None
    try:
        # Extract the currency symbol from the string
        currency_symbol = row['fees'][0]

        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(row['fees'][1:].replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)

    except Exception as e:
       return None


In [ ]:
# Apply the conversion function to the 'fees' column
raw_fees['fees'] = raw_fees.apply(convert_to_common_currency, axis=1)

# Rename the column to indicate the common currency
common_currency_code = 'USD'
raw_fees.rename(columns={'fees': f'fees ({common_currency_code})'}, inplace=True)

In [ ]:
raw_fees.head()

,fees (USD)
0,35778.51
1,NaN
2,NaN
3,898.51
4,NaN


sustituing and renaming the new column with the appropriate currance values

In [ ]:
processed_df['fees']= raw_fees['fees (USD)']
processed_df.rename(columns={'fees': 'fees (USD)'}, inplace=True)


In [ ]:
processed_df.head()

#[2.1]

##[2.1.1]

In [ ]:
vocabulary = set()

In [ ]:
processed_df['description']= processed_df.description.apply(lambda row: word_tokenize(row))

In [ ]:
processed_df.description.head()

0    [7, million, peopl, estim, die, everi, year, d...
1    [analyt, toxicolog, msc, uniqu, studi, cours, ...
2    [comput, scienc, foundat, mani, excit, cutting...
3    [polit, uncertainti, financ, recruit, demands,...
4    [thi, master, focus, linguist, research, help,...
Name: description, dtype: object

In [ ]:
processed_df.description.apply(lambda row: [vocabulary.add(word) for word in row])


0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5974    [None, None, None, None, None, None, None, Non...
5975    [None, None, None, None, None, None, None, Non...
5976    [None, None, None, None, None, None, None, Non...
5977    [None, None, None, None, None, None, None, Non...
5978    [None, None, None, None, None, None, None, Non...
Name: description, Length: 5979, dtype: object

In [ ]:
vocabulary

{'community-bas',
 'dies',
 'surfac',
 'fruit',
 'lipid',
 'transformed',
 'cortex',
 'novic',
 'ranking',
 'unless',
 'nowaday',
 'biophys',
 'explor',
 'diversified',
 '-accredited',
 'università',
 'hydrologist',
 'wurd',
 'multi',
 'defences',
 'anna',
 'populations',
 'tak',
 'demystifi',
 'accuraci',
 'loss',
 'king',
 'criminalis',
 '98',
 'elig',
 'multidimension',
 'stars',
 'automotive',
 'taskforc',
 'dual-profil',
 'memori',
 'leagu',
 'thet',
 'wherev',
 'emin',
 'low-effort',
 'texts',
 'non-stat',
 'covers',
 'apparatu',
 'nantes',
 'weath',
 'mammograph',
 'pgdip/msc',
 'incred',
 'mahsr',
 'smart',
 'reporting',
 'columbus',
 'magnet',
 'equipment/devic',
 'procedures',
 'minim',
 'sabancı',
 'estonian',
 '»',
 'adept',
 'galleri',
 'glaciers',
 'seriously',
 'humankind',
 'techno-econom',
 'equine',
 'calcul',
 'affect',
 'agro-',
 'boast',
 'festivals',
 'general',
 'immunotherapi',
 'non-thesi',
 'evaluation/develop',
 'multidisciplinari',
 'paradigms',
 'diplomat',

 Assign unique ID to each term in the vocabulary

In [ ]:
vocabulary_list = list(vocabulary)
vocabulary_dict = {word: index for index, word in enumerate(vocabulary_list)}
vocabulary_df=pd.DataFrame(list(vocabulary_dict.items()), columns=['Word', 'Id'])
print(vocabulary_df.head())

            Word  Id
0  community-bas   0
1           dies   1
2         surfac   2
3          fruit   3
4          lipid   4


save it

In [ ]:
vocabulary_df.to_csv('/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.csv', index=False, header=False)
#remove header to keep the name of the columns

In [ ]:
# Store in a json llike a pandas dataframe
vocabulary_df.to_json('/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.json', orient='records')

In [ ]:
# Write the dictionary to the JSON file directly
with open('/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.json', 'w') as jsonfile:
    json.dump(vocabulary_dict, jsonfile)

In [ ]:
vocabulary_reverse = vocabulary_df.copy()
print(vocabulary_reverse.head())

            Word  Id
0  community-bas   0
1           dies   1
2         surfac   2
3          fruit   3
4          lipid   4


In [ ]:
from tqdm import tqdm
tqdm.pandas()
vocabulary_reverse['reverse'] = vocabulary_reverse.Word.progress_apply(lambda item: list(processed_df.loc[processed_df.description.apply(lambda row: item in row)].index))

100%|██████████| 12547/12547 [02:17<00:00, 91.25it/s] 


In [ ]:
vocabulary_reverse.head()

,Word,Id,reverse
0,community-bas,0,"[697, 1792]"
1,dies,1,"[4422, 4429]"
2,surfac,2,"[102, 2001, 2205, 2764, 2791, 3272, 3830, 4290..."
3,fruit,3,[3883]
4,lipid,4,[5925]


In [ ]:
vocabulary_reverse[vocabulary_reverse['reverse'].apply(lambda x: len(x) == 0)]

,Word,Id,reverse


In [ ]:
vocabulary_reverse[vocabulary_reverse['Word'] == "'all"]


,Word,Id,reverse
643,'all,643,[3871]


#### creating a dictionary

In [ ]:
inverted_index = vocabulary_reverse.set_index('Id')['reverse'].to_dict()
count = 0
for key, value in inverted_index.items():
    if count < 5:
        print(f"Key: {key}, Value: {value}")
        count += 1
    else:
        break

Key: 0, Value: [697, 1792]
Key: 1, Value: [4422, 4429]
Key: 2, Value: [102, 2001, 2205, 2764, 2791, 3272, 3830, 4290, 4425, 4538, 5203, 5944]
Key: 3, Value: [3883]
Key: 4, Value: [5925]


In [ ]:
with open('/content/drive/MyDrive/HM3-ADM/HW3_ADM/inverted_index.json', 'w') as file:
    json.dump(inverted_index, file)

##[2.1.2]

selecting only the rows we need to return as output

In [ ]:
df_query = df[['courseName','universityName','description','url']].copy()
df_query.head()

,courseName,universityName,description,url
0,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
2,Applied Computer Science and Artificial Inte...,University of Bradford,Computer science is the foundation of many exc...,https://www.findamasters.com/masters-degrees/c...
3,Applied Economics (Banking and Financial Mar...,University of Bath,From political uncertainty to finance and recr...,https://www.findamasters.com/masters-degrees/c...
4,Applied Linguistics - MSc,University of Glasgow,This Masters focuses on how linguistic researc...,https://www.findamasters.com/masters-degrees/c...


In [ ]:
df_query.shape

(5979, 4)

writing a function which extract all documents were all words are met

In [ ]:
def engine(query):
    """
    Given a query made up by multiple word it returns the documents were ALL the word are found
    """
    doc_set_indexes = []
    words_in_query = query.split()

    for word in words_in_query:
        # Stemming the word
        stemmed_word = stemmer.stem(word)

        # Check if the stemmed word exists in the 'Word' column after applying stemming
        if stemmed_word in vocabulary_reverse['Word'].apply(stemmer.stem).values:
            # Get the document set indexes for the stemmed word
            indexes_for_word = vocabulary_reverse[vocabulary_reverse['Word'].apply(lambda x: x == stemmed_word)]['reverse'].values

            # Flatten the lists in 'reverse' column
            flattened_indexes = [item for sublist in indexes_for_word for item in sublist]

            # Append the flattened document set indexes to the list
            doc_set_indexes.append(flattened_indexes)
           # print(doc_set_indexes)

        else:
            print(f"Stemmed word '{stemmed_word}' not found in vocabulary_reverse")

    # Find the intersection of all document sets
    selected_doc = list(set.intersection(*map(set, doc_set_indexes)))

    # Select rows using iloc
    selected_rows = df.iloc[selected_doc]

    return selected_rows


In [ ]:
engine('advance knowledge')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
1,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) \nInternational: £34,750 (...",MSc,1 year full time,Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
8,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2056,Engineering with Management - MSc,University of Bristol,Faculty of Engineering,Full time,The MSc in Engineering with Management is desi...,September,Please see the university website for further ...,MSc,1 year full-time,Bristol,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2069,Enhanced Radiotherapy and Oncology Practice ...,Sheffield Hallam University,Postgraduate Courses,Part time,Enhance your knowledge of specific radiotherap...,"September, January",For part-time study the fee will be calculated...,MSc,3 years part time distance learning,Sheffield,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Engineering Management (MSc),University of Gloucestershire,School of Computing and Engineering,Full time,This course will equip you with the necessary ...,September,Please see the university website for further ...,MSc,See website for details,Gloucester,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2032,Engineering Management MSc,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,The MSc Engineering Management programme will ...,"September, January",Please see the university website for further ...,MSc,1 year Full time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2034,Engineering Management MSc,Middlesex University,Faculty of Science and Technology,Full time,The career of an engineering project manager i...,October,"UK students \nFull-time students: £10,500 \nPa...",MSc,1 year full-time; 2 years part-time,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4083,M2 Research in Management and Innovation,Université Côte d’Azur,UCA International MSc Program,Full time,"Our modern industry structures, value chains a...",September,The tuition currently stands at €243 as well a...,MSc,2 years,Nice,On Campus,France,https://www.findamasters.com/masters-degrees/c...


# [2.2] Conjunctive query & Ranking score

## [2.2.1] Inverted index

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
print(processed_df.loc[2, 'description'])

['comput', 'scienc', 'foundat', 'mani', 'excit', 'cutting-edg', 'topics', 'technolog', 'applications', 'thi', 'cover', 'ai', 'big', 'data', 'cybersecurity', 'span', 'almost', 'everi', 'aspect', 'lives', 'smartphon', 'app', 'development', 'self-driv', 'assist', 'revolutionis', 'safeti', 'effici', 'transport', 'systems', 'du', 'pandemic', 'computer-rel', 'skill', 'becom', 'import', 'requir', 'work', 'remotely', 'convert', 'comput', 'scienc', 'could', 'develop', 'problem-solv', 'practic', 'skills', 'also', 'establish', 'specialist', 'knowledg', 'tool', 'techniqu', 'creat', 'intellig', 'algorithm', 'software']


In [ ]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(processed_df['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=processed_df.index, columns=tfidf.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
print(tfidf_data.head())

     '  'all  'appli  'bench  'big  'chip  'credits   'd  'disability  'educ  \
0  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
1  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
2  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
3  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
4  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   

   ...  ﻿over  ﻿prove  ﻿regist  ﻿th  ﻿the  ﻿theoret  ﻿thi  ﻿want   ﻿y   ﻿﻿  
0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
1  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
2  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
3  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
4  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  

[5 rows x 12547 columns]


### Controllo la lunghezza per vedere se ci sono tutte le parole: ok (?)

In [ ]:
len(tfidf_data)

5979

### Questi due codici successivi solo per vedere se ci sono elementi diversi da 0

In [ ]:
tfidf_data.loc[tfidf_data.regist > 0, ['prove']]

,prove
94,0.0
116,0.0
118,0.0
121,0.0
148,0.0
...,...
5924,0.0
5928,0.0
5939,0.0
5965,0.0


In [ ]:
tfidf_data.loc[tfidf_data.regist > 0]

,'all,'appli,'bench,'big,'chip,'credits,'d,'disability,'educ,'entri,...,﻿over,﻿prove,﻿regist,﻿th,﻿the,﻿theoret,﻿thi,﻿want,﻿y,﻿﻿
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Costruisco un dizionario

In [ ]:
inverted_index_2 = {}
for word in tfidf_data.columns:
    # Filtraggio dei documenti in cui la parola ha un TFIDF>0: non so se vogliamo farlo seno si leva
    word_docs = tfidf_data.index[tfidf_data[word] > 0].tolist()
    word_doc_scores = [(doc, tfidf_data.loc[doc, word]) for doc in word_docs]
    inverted_index_2[word] = word_doc_scores


### Stampiamo per vedere che esce

In [ ]:
count = 0
for word, doc_scores in inverted_index_2.items():
    print(f"Word: {word}")
    print("Document Scores:")
    print(doc_scores)
    count += 1
    if count == 10:
        break

Word: 'all
Document Scores:
[(3871, 0.1849258796135223)]
Word: 'appli
Document Scores:
[(2325, 0.1722113554646612)]
Word: 'bench
Document Scores:
[(333, 0.3466489203597607)]
Word: 'big
Document Scores:
[(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)]
Word: 'chip
Document Scores:
[(1902, 0.1992739368962018)]
Word: 'credits
Document Scores:
[(2317, 0.18303193893688977), (5926, 0.08382342995897132)]
Word: 'd
Document Scores:
[(2160, 0.194367236072988), (4863, 0.14435725548964343)]
Word: 'disability
Document Scores:
[(4783, 0.17222181095019545)]
Word: 'educ
Document Scores:
[(2618, 0.2647135148766833)]
Word: 'entri
Document Scores:
[(641, 0.20952396586542038)]


## Ma noi dobbiamo avere l'id della parola come chiave non la parola stessa e questo id lo prendiamo da vocabulary e creiamo un nuovo dizionario che è quello richiesto dalla traccia

In [ ]:
new_inverted_index = {}
for word, doc_scores in inverted_index_2.items():
    word_id = vocabulary_df[vocabulary_df['Word'] == word]['Id'].values
    if len(word_id) > 0:
        word_id = word_id[0]
        new_inverted_index[word_id] = doc_scores


### Stampiamo

In [ ]:
count = 0
for word, doc_scores in new_inverted_index.items():
    print(f"Word: {word}")
    print("Document Scores:")
    print(doc_scores)
    count += 1
    if count == 10:
        break

Word: 643
Document Scores:
[(3871, 0.1849258796135223)]
Word: 4806
Document Scores:
[(2325, 0.1722113554646612)]
Word: 8232
Document Scores:
[(333, 0.3466489203597607)]
Word: 3792
Document Scores:
[(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)]
Word: 7472
Document Scores:
[(1902, 0.1992739368962018)]
Word: 8815
Document Scores:
[(2317, 0.18303193893688977), (5926, 0.08382342995897132)]
Word: 8418
Document Scores:
[(2160, 0.194367236072988), (4863, 0.14435725548964343)]
Word: 4772
Document Scores:
[(4783, 0.17222181095019545)]
Word: 8468
Document Scores:
[(2618, 0.2647135148766833)]
Word: 10309
Document Scores:
[(641, 0.20952396586542038)]


### Stampiamo le prime 10 chiavi per vedere se è come è richiesto

In [ ]:
# Stampa solo le prime n chiavi del nuovo dizionario
n = 10  # Imposta il numero di chiavi da stampare

# Stampare le prime n chiavi con i relativi valori
partial_inverted_index = {k: new_inverted_index[k] for k in list(new_inverted_index.keys())[:n]}
print(partial_inverted_index)

{7193: [(3244, 0.2724922372887123)], 3412: [(3871, 0.1849258796135223)], 2879: [(2325, 0.1722113554646612)], 2571: [(333, 0.3466489203597607)], 4592: [(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)], 9166: [(1902, 0.1992739368962018)], 1753: [(2317, 0.18303193893688977), (5926, 0.08382342995897132)], 10075: [(2160, 0.194367236072988), (4863, 0.14065952388070307)], 5828: [(4783, 0.17222181095019545)], 4932: [(2618, 0.2647135148766833)]}


### Nei prossimi codici analizzo la chiave 0 quindi la parola che ha id 0 che come possiamo vedere anche sopra corrisponde con questa che poi stampo nel terzo codice. L'ho fatto per confrontare i due inverted_index perche per ogni chiave i documenti dovrebbero essere gli stessi , ovviamente a questo nuovo poi a ogni documento è associato l'indice. L'ultimo codice l'ho fatto perche a me esce un 3201 in piu nel nuovo inverted_index e effettivamente dovrebbe uscire anche all'inverted_index ma almeno a me non esce (poi magari con le cose runnate tue si trova)

In [ ]:
key_to_print = 0
if key_to_print in new_inverted_index:
    print(new_inverted_index[key_to_print])
else:
    print("La chiave specificata non è presente nel dizionario.")

[(697, 0.15539268962926714), (1792, 0.33655643188743606)]


In [ ]:
key_to_print = 0
if key_to_print in inverted_index:
    print(inverted_index[key_to_print])
else:
    print("La chiave specificata non è presente nel dizionario.")

[697, 1792]


In [ ]:
target_id = 0
word = vocabulary_df.loc[vocabulary_df['Id'] == target_id, 'Word'].values[0]
print(word)

community-bas


In [ ]:
target_index=3201  # Indice desiderato
# Verifica se l'indice è presente nel DataFrame
if target_index in processed_df.index:
    description_value = processed_df.loc[target_index, 'description']
    print(description_value)
else:
    print(f"L'indice {target_index} non è presente nel DataFrame.")

['programm', 'description', 'human', 'skelet', 'remain', 'direct', 'evid', 'past', 'lifeway', 'scientif', 'investig', 'give', 'uniqu', 'insight', 'human', 'history', 'bioarchaeology', 'also', 'known', 'osteoarchaeolog', 'studi', 'archaeolog', 'human', 'remains', 'subdisciplin', 'biolog', 'anthropology', 'excit', 'interdisciplinari', 'field', 'draw', 'varieti', 'techniques', 'rang', 'visual', 'examin', 'whole', 'skeleton', 'biomolecular', 'analysi', 'small', 'bone', 'samples', 'demograph', 'shifts', 'environment', 'changes', 'migrations', 'lifestyles', 'numer', 'diseas', 'impact', 'violenc', 'conflict', 'leav', 'trace', 'skeleton', 'thi', 'msc', 'provid', 'skill', 'requir']
